In [1]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [2]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'


In [3]:
import string
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import numpy as np
import pygwalker as pyg
import googletrans
from googletrans import Translator
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tag import pos_tag, pos_tag_sents
from spacy.pipeline import Sentencizer


In [4]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_dropped.pkl').reset_index(drop=True)


In [5]:
len(df_jobs)


62577

In [6]:
df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
    lambda job_description: ' '.join(job_description.split('/')) if '/' in job_description else job_description
)


In [7]:
abb_dict = {
    r'incl\.': 'including', 
    r'e\.g\.': 'for example', 
    r'e\.g': 'for example', 
    r'etc\.': 'et cetera', 
}


In [8]:
str_fix_incl = 'Apply appropriate and effective communication methods to senior management and important stakeholders'
str_fix_eg = 'Partner with Procurement in order to manage suppliers for the projects & programs in scope'



In [9]:
df_jobs['Job Description'] = df_jobs['Job Description'].replace(abb_dict, regex=True)

In [10]:
df_jobs['Job Description'][df_jobs['Job Description'].str.contains(str_fix_incl)]


35      Purpose\n\nAs a Studio A Director\n\nSpecial P...
3423    Purpose:\nAs a Manager Digital Strategy and Pr...
Name: Job Description, dtype: object

In [11]:
re.findall(str_fix_eg, df_jobs['Job Description'][35])

['Partner with Procurement in order to manage suppliers for the projects & programs in scope']

In [12]:
list(re.finditer(str_fix_eg, df_jobs['Job Description'][35]))

[<re.Match object; span=(1461, 1551), match='Partner with Procurement in order to manage suppl>]

In [13]:
df_jobs['Job Description'][35][1461:1580]

'Partner with Procurement in order to manage suppliers for the projects & programs in scope, for example interviews, sta'

In [14]:
# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('incl.', 'including')
# )

# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('e.g.', 'e.g')
# )

# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('e.g', 'for example')
# )

# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('etc.', 'et cetera')
# )


In [15]:
# df_jobs = df_jobs[:100]

In [16]:
# df_jobs = df_jobs.sample(n=100, random_state=1)

In [17]:
# df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_glob_paths_10.pkl').reset_index(drop=True)


In [18]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62577 entries, 0 to 62576
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Search Keyword     62577 non-null  object 
 1   Platform           62577 non-null  object 
 2   Job ID             62577 non-null  object 
 3   Job Title          62577 non-null  object 
 4   Company Name       62574 non-null  object 
 5   Location           62577 non-null  object 
 6   Job Description    62577 non-null  object 
 7   Rating             3975 non-null   float64
 8   Employment Type    61995 non-null  object 
 9   Company URL        59263 non-null  object 
 10  Job URL            62577 non-null  object 
 11  Job Age            62577 non-null  object 
 12  Job Age Number     62577 non-null  object 
 13  Collection Date    62577 non-null  object 
 14  Data Row           58599 non-null  float64
 15  Tracking ID        58599 non-null  object 
 16  Industry           591

In [19]:
df_jobs.columns


Index(['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name',
       'Location', 'Job Description', 'Rating', 'Employment Type',
       'Company URL', 'Job URL', 'Job Age', 'Job Age Number',
       'Collection Date', 'Data Row', 'Tracking ID', 'Industry', 'Job Date',
       'Type of ownership'],
      dtype='object')

In [20]:
len(df_jobs.loc[(df_jobs['Job Description'].str.contains(str_fix_incl)) | (df_jobs['Job Description'].str.contains(str_fix_eg))])


2

In [21]:
# job_descriptions = list(set(df_jobs['Job Description'].to_list()))


In [22]:
job_descriptions = list(
    set(
        df_jobs['Job Description'].loc[
            (df_jobs['Job Description'].str.contains(str_fix_incl)) |
            (df_jobs['Job Description'].str.contains(str_fix_eg))
        ].to_list()
    )
)


In [23]:
len(job_descriptions)


2

In [24]:
job_descriptions[0][1500:2000]

'and important stakeholders throughout the project lifecycle.\nAs conflicts and escalations arise within projects, identify solutions and drive their resolution in a timely and appropriate manner.\nManage change within projects and ensure changes are smoothly and successfully implemented to achieve lasting benefits.\nProject Controlling\nManage project controlling as an independent element to ensure that actual project costs are in line the committed budget.\nConduct engagement reviews. Verify complia'

In [25]:
from gensim.utils import save_as_line_sentence, simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string, preprocess_documents

simple_tokenization = preprocess_documents(job_descriptions)

In [26]:
simple_tokenization

[['purpos',
  'manag',
  'digit',
  'strategi',
  'program',
  'adida',
  'market',
  'ecom',
  'organ',
  'task',
  'manag',
  'deliv',
  'small',
  'digit',
  'project',
  'signific',
  'rfc',
  'sub',
  'element',
  'mid',
  'scale',
  'digit',
  'project',
  'project',
  'scope',
  'local',
  'market',
  'level',
  'brand',
  'function',
  'object',
  'conduct',
  'project',
  'boundari',
  'time',
  'cost',
  'qualiti',
  'focus',
  'consum',
  'expect',
  'requir',
  'plan',
  'execut',
  'track',
  'project',
  'cooper',
  'respect',
  'function',
  'project',
  'team',
  'member',
  'exampl',
  'expect',
  'role',
  'addition',
  'focus',
  'resourc',
  'manag',
  'coordin',
  'activ',
  'act',
  'central',
  'interfac',
  'local',
  'stakehold',
  'kei',
  'respons',
  'scope',
  'manag',
  'deliv',
  'small',
  'digit',
  'project',
  'local',
  'market',
  'level',
  'project',
  'program',
  'deliveri',
  'deliv',
  'project',
  'end',
  'end',
  'successfulli',
  'ideal',


In [27]:
# Load NLK
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer, LancasterStemmer

nltk_path = f'{llm_path}/nltk'
nltk.data.path.append(nltk_path)

nltk.download('words', download_dir = nltk_path)
nltk.download('stopwords', download_dir = nltk_path)
nltk.download('punkt', download_dir = nltk_path)
nltk.download('averaged_perceptron_tagger', download_dir = nltk_path)

stop_words = set(stopwords.words('english'))
punctuations = list(string.punctuation)


[nltk_data] Downloading package words to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package averaged_perceptron_

In [28]:
# pattern = r'[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?<=[a-z])(?=[A-Z])'
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'

pattern_numbers = r'[^a-zA-Z0-9]'

In [29]:
# nltk_sentencized = []

# for job_description in job_descriptions:
#     for sentence in sent_tokenize(job_description):
#         for sent in re.split(pattern, sentence):
#             nltk_sentencized.append(sent)

In [30]:
nltk_sentencized = [
    sent 
    for job_description in job_descriptions 
    for sentence in sent_tokenize(job_description) 
    for sent in re.split(pattern, sentence)
    if len(sent) != 0
]

In [31]:
nltk_sentencized_lower = [
    str(sent.strip().lower()) 
    for sent in nltk_sentencized
]

In [32]:
len(nltk_sentencized)

133

In [33]:
len(nltk_sentencized_lower)

133

In [34]:
for idx, sent in enumerate(nltk_sentencized):
    if str_fix_eg in sent:
        print(idx)

84


In [35]:
next(idx for idx, sent in enumerate(nltk_sentencized) if str_fix_eg in sent)

84

In [36]:
# str_fix = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'

for idx, sent in enumerate(nltk_sentencized):
    if str_fix_incl.split('\(')[0] in sent:
        print(idx)

18
87


In [37]:
nltk_sentencized[84:95]

['Partner with Procurement in order to manage suppliers for the projects & programs in scope, for example interviews, staffing, supplier costs.',
 'Communication   stakeholder management',
 'Act as the key contact person for strategic stakeholders.',
 'Apply appropriate and effective communication methods to senior management and important stakeholders (including vendor) throughout the project lifecycle.',
 'As conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.',
 'Drive change management activities for respective projects & programs and ensure changes are smoothly and successfully implemented to achieve lasting benefits.',
 'Project Controlling',
 'Manage project controlling as an independent element to ensure that actual project costs are in line the committed budget.',
 'Lead or conduct engagement reviews.',
 'Verify implementation of quality assurance procedures.',
 'Ensure project & program delivery aga

In [38]:
# sentences split on e.g. and incl.
nltk_sentencized[
    next(idx for idx, sent in enumerate(nltk_sentencized) if str_fix_eg in sent):next(idx for idx, sent in enumerate(nltk_sentencized) if str_fix_incl.split('\(')[0] in sent)+2
]


[]

In [39]:
for idx, sent in enumerate(nltk_sentencized):
    if 'Power' in sent:
        print(idx)

62
129


In [40]:
next(idx for idx, sent in enumerate(nltk_sentencized) if 'Power' in sent)

62

In [41]:
nltk_sentencized[next(idx for idx, sent in enumerate(nltk_sentencized) if 'Power' in sent)]

'Strong MS-Office skills (Word, Excel, PowerPoint)'

In [42]:
for idx, sentence in enumerate(nltk_sentencized[84:95]):
    print(f'Sentence {idx+1}: {sentence}\n')

Sentence 1: Partner with Procurement in order to manage suppliers for the projects & programs in scope, for example interviews, staffing, supplier costs.

Sentence 2: Communication   stakeholder management

Sentence 3: Act as the key contact person for strategic stakeholders.

Sentence 4: Apply appropriate and effective communication methods to senior management and important stakeholders (including vendor) throughout the project lifecycle.

Sentence 5: As conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.

Sentence 6: Drive change management activities for respective projects & programs and ensure changes are smoothly and successfully implemented to achieve lasting benefits.

Sentence 7: Project Controlling

Sentence 8: Manage project controlling as an independent element to ensure that actual project costs are in line the committed budget.

Sentence 9: Lead or conduct engagement reviews.

Sentence 10: Veri

In [43]:
# nltk_tokenized = []

# for job_sentence in nltk_sentencized:
#     for token in word_tokenize(job_sentence):
#         if len(token) != 0 and token != '...' and token.lower() not in set(stopwords.words('english')) and token.lower() not in list(string.punctuation):
#             nltk_tokenized.append(str(token.strip().lower())))


In [44]:
nltk_tokenized = [
    str(token.strip().lower()) 
    for job_sentence in nltk_sentencized 
    for token in word_tokenize(job_sentence) 
    if len(token) != 0 
    and token != '...' 
    and not token.lower() in set(stopwords.words('english')) 
    and not token.lower() in list(string.punctuation) 
]

In [45]:
'...' in nltk_tokenized

False

In [46]:
len(nltk_tokenized)

915

In [47]:
nltk_token_tags = pos_tag(nltk_tokenized)

In [48]:
len(nltk_token_tags)

915

In [49]:
nltk_token_tags[1][1]

'NN'

In [50]:
# https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

In [51]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([token])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [52]:
import unicodedata

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

nltk_lemmas = []
nltk_stems = []

for token in nltk_tokenized:
    token = unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    token_lemma = lemmatizer.lemmatize(token)
    token_stem = stemmer.stem(token)
    token_pos = get_wordnet_pos(token)
    token_pos_lemma = lemmatizer.lemmatize(token, token_pos)
    print(f'Token: {token} | Stem: {token_stem} | Lemma: {token_lemma} | POS Lemma: {token_pos_lemma}')
    nltk_lemmas.append(token_pos_lemma)
    nltk_stems.append(token_stem)


Token: purpose | Stem: purpos | Lemma: purpose | POS Lemma: purpose
Token: manager | Stem: manag | Lemma: manager | POS Lemma: manager
Token: digital | Stem: digit | Lemma: digital | POS Lemma: digital
Token: strategy | Stem: strategi | Lemma: strategy | POS Lemma: strategy
Token: programs | Stem: program | Lemma: program | POS Lemma: program
Token: adidas | Stem: adida | Lemma: adidas | POS Lemma: adidas
Token: market | Stem: market | Lemma: market | POS Lemma: market
Token: ecom | Stem: ecom | Lemma: ecom | POS Lemma: ecom
Token: organization | Stem: organ | Lemma: organization | POS Lemma: organization
Token: task | Stem: task | Lemma: task | POS Lemma: task
Token: manage | Stem: manag | Lemma: manage | POS Lemma: manage
Token: deliver | Stem: deliv | Lemma: deliver | POS Lemma: deliver
Token: small | Stem: small | Lemma: small | POS Lemma: small
Token: digital | Stem: digit | Lemma: digital | POS Lemma: digital
Token: projects | Stem: project | Lemma: project | POS Lemma: project
T

In [53]:
# Function to make a list of punctuations that determine sentence boundry, i.e., split characters
def make_custom_punct_chars(main_punct_chars = [':', '|'], repeated_punct_chars = ['\n', ',']):
    custom_punct_chars = []
    temp_multi = []
    temp_spaced = []

    for punct_char in main_punct_chars:
        custom_punct_chars+= f'{punct_char}', f'{punct_char} '

    for idx in range(4):
        for punct_char in repeated_punct_chars:
            temp_multi.append(f'{punct_char}'*int(idx+1))
            temp_spaced.append(f'{punct_char} '*int(idx+1))

    for multi, spaced in zip(temp_multi, temp_spaced):
        custom_punct_chars+= multi, spaced

    custom_punct_chars.remove(',')
    custom_punct_chars.remove(', ')

    return custom_punct_chars


In [54]:
custom_punct_chars = make_custom_punct_chars()

In [55]:
custom_punct_chars

[':',
 ': ',
 '|',
 '| ',
 '\n',
 '\n ',
 '\n\n',
 '\n \n ',
 ',,',
 ', , ',
 '\n\n\n',
 '\n \n \n ',
 ',,,',
 ', , , ',
 '\n\n\n\n',
 '\n \n \n \n ',
 ',,,,',
 ', , , , ']

In [56]:
# Set up Spacy
import spacy
from spacy.symbols import NORM, ORTH, LEMMA, POS

nlp = spacy.load('en_core_web_sm')


In [57]:
sentencizer = nlp.add_pipe('sentencizer')
sentencizer.punct_chars.update(custom_punct_chars)


In [58]:
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    with open(f'{data_dir}punctuations.txt', 'wb') as f:
        pickle.dump(sentencizer.punct_chars, f)

In [59]:
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_char = pickle.load(f)

In [60]:
# special_cases = {":)": [{"ORTH": ":)"}]}
# prefix_re = re.compile(r'''^[\\[\\("']''')
# suffix_re = re.compile(r'''[\\]\\)"']$''')
# infix_re = re.compile(r'''[-~]''')
# simple_url_re = re.compile(r'''^https?://''')

# def custom_tokenizer(nlp):
#     return Tokenizer(nlp.vocab, rules=special_cases,
#                                 prefix_search=prefix_re.search,
#                                 suffix_search=suffix_re.search,
#                                 infix_finditer=infix_re.finditer,
#                                 url_match=simple_url_re.match)

In [61]:
# nlp.tokenizer.add_special_case('incl', [{ORTH: u'incl', NORM: u'include', LEMMA: u'include', POS: u'VERB'}])


In [62]:
from spacy.symbols import ORTH, LEMMA

special_cases_dict = {
    'incl.': [{65: 'incl', 67: 'including'}],
    'incl. ': [{65: 'incl', 67: 'including'}],
    '(incl.': [{65: 'incl', 67: 'including'}],
    'etc.': [{65: 'etc', 67: 'et cetera'}],
    'etc. ': [{65: 'etc', 67: 'et cetera'}],
    'e.g.': [{65: 'e.g', 67: 'for example'}],
    'e.g. ': [{65: 'e.g', 67: 'for example'}],
}

nlp.tokenizer.rules.update(special_cases_dict)


In [63]:
nlp.tokenizer.rules['e.g.']

[{65: 'e.g', 67: 'for example'}]

In [64]:
# # Spacy sentencizer
# spacy_sentencized = []

# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     for job_description in job_descriptions:
#         for sentence in nlp(job_description).sents:
#             for sent in re.split(pattern, sentence.text):
#                 if len(sent) != 0:
#                     spacy_sentencized.append(sent)

In [65]:
# Spacy sentencizer
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    spacy_sentencized = [
        sent 
        for job_description in job_descriptions 
        for sentence in nlp(job_description).sents 
        for sent in re.split(pattern, sentence.text) 
        if len(sent) != 0 
    ]
    
    spacy_sentencized_lower = [
        str(sent.strip().lower()) 
        for sent in spacy_sentencized
    ]


In [66]:
# # Spacy sentencizer
# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     spacy_sentencized = [
#         re.split(pattern, sentence.text) 
#         for job_description in job_descriptions 
#         for sentence in nlp(job_description).sents 
# #         for sent in re.split(pattern, sentence.text) 
#         if len(sentence) != 0 
#     ]



In [106]:
spacy_sentencized[23]

'Conduct engagement reviews.'

In [68]:
# # Spacy tokenizer
# spacy_tokenized = []

# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     for job_sentence in spacy_sentencized:
#         for token in nlp.tokenizer(job_sentence):
#             if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars:
#                 spacy_tokenized.append(token.text)

In [69]:
type(spacy_sentencized[0])

str

In [70]:
len(spacy_sentencized)

135

In [72]:
for idx, sent in enumerate(spacy_sentencized):
    if str_fix_eg in sent:
        print(idx)

86


In [73]:
next(idx for idx, sent in enumerate(spacy_sentencized) if str_fix_eg in sent)

86

In [74]:
# str_fix = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'


for idx, sent in enumerate(spacy_sentencized):
    if str_fix_incl.split('\(')[0] in sent:
        print(idx)

18
89


In [75]:
next(idx for idx, sent in enumerate(spacy_sentencized) if str_fix_incl.split('\(')[0] in sent)

18

In [76]:
spacy_sentencized[18:25]

['Apply appropriate and effective communication methods to senior management and important stakeholders throughout the project lifecycle.',
 'As conflicts and escalations arise within projects, identify solutions and drive their resolution in a timely and appropriate manner.',
 'Manage change within projects and ensure changes are smoothly and successfully implemented to achieve lasting benefits.',
 'Project Controlling',
 'Manage project controlling as an independent element to ensure that actual project costs are in line the committed budget.',
 'Conduct engagement reviews.',
 'Verify compliance with quality assurance procedures.']

In [77]:
for idx, sent in enumerate(spacy_sentencized):
    if 'Power' in sent:
        print(idx)

63
131


In [78]:
next(idx for idx, sent in enumerate(spacy_sentencized) if 'Power' in sent)

63

In [79]:
spacy_sentencized[63]

'Strong MS-Office skills (Word, Excel, PowerPoint)'

In [80]:
for idx, sentence in enumerate(spacy_sentencized[18:25]):
    print(f'Sentence {idx+1}: {sentence}\n')

Sentence 1: Apply appropriate and effective communication methods to senior management and important stakeholders throughout the project lifecycle.

Sentence 2: As conflicts and escalations arise within projects, identify solutions and drive their resolution in a timely and appropriate manner.

Sentence 3: Manage change within projects and ensure changes are smoothly and successfully implemented to achieve lasting benefits.

Sentence 4: Project Controlling

Sentence 5: Manage project controlling as an independent element to ensure that actual project costs are in line the committed budget.

Sentence 6: Conduct engagement reviews.

Sentence 7: Verify compliance with quality assurance procedures.



In [81]:
# # Spacy tokenizer
# spacy_tokenized = []

# for job_sentence in spacy_sentencized:
# #         doc = nlp.tokenizer(job_sentence)
#     spacy_tokenized.extend(
#         [
#             token.text for token in nlp.tokenizer(job_sentence) 
#             if token.text not in custom_punct_chars
#             and not token.is_stop 

#         ]
#     )

In [82]:
type(spacy_sentencized[0])

str

In [83]:
spacy_sentencized[6]

'Key Responsibilities'

In [84]:
# Spacy tokenizer

spacy_tokenized = [
    str(token.text.strip().lower()) 
    for job_sentence in spacy_sentencized 
    for token in nlp.tokenizer(job_sentence) 
    if len(token) != 0 
    and not token.is_stop 
    and not token.is_punct 
    and not token.text in custom_punct_chars
]


In [85]:
len(spacy_tokenized)

910

In [86]:
spacy_tokenized[:10]

['purpose',
 'manager',
 'digital',
 'strategy',
 'programs',
 'adidas',
 'market',
 'ecom',
 'organization',
 'task']

In [87]:
# nlp.get_pipe('attribute_ruler').add([[{"TEXT":"Angeltown"}]],{"LEMMA":"San Fransisco"})

In [88]:
# # Spacy pos tagging
# nlp_stemmer = LancasterStemmer()
# nlp_stemmer = PorterStemmer()

nlp_token_tags = []
nlp_lemmas = []
nlp_stems = []

for job_description in job_descriptions:
    for token in nlp(job_description):
        if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars:
            nlp_token_tags.append(tuple([token.text, token.tag_]))
            nlp_lemmas.append(token.lemma_)
            nlp_stems.append(stemmer.stem(token.text))
#         for sentence in doc.sents:
#             for sent in re.split(pattern, sentence.text):
#                 if len(sent) != 0:
#                     spacy_sentencized.append(sent)

In [89]:
# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     nlp_token_tags = [
#         tuple([token.text, token.tag_]) 
#         for job_description in job_descriptions 
#         for token in nlp(job_description) 
#         if len(token) != 0 
#         and not token.is_stop 
#         and not token.is_punct 
#         and not token.text in custom_punct_chars
#     ]

In [90]:
def get_word_num_and_frequency(row, text_col):

    row['num_words'] = len(str(row[f'{text_col}']).split())
    row['num_unique_words'] = len(set(str(row[f'{text_col}']).split()))
    row['num_chars'] = len(str(row[f'{text_col}']))
    row['num_punctuations'] = len([c for c in str(row[f'{text_col}']) if c in string.punctuation])

    return row


In [105]:
for sent in spacy_sentencized:
    num_words = len(sent.split())
    num_unique_words = len(set(sent.split()))
    num_chars = len(sent)
    num_punctuations = len([c for c in sent if c in string.punctuation])

In [ ]:
def get_sentiment(df_jobs_to_be_processed, text_col, algo='vader', sentiment_range=(-1,1)):

    ## calculate sentiment
    if algo == 'vader':
        df_jobs_to_be_processed['sentiment'] = df_jobs_to_be_processed[text_col].progress_apply(lambda x: SentimentIntensityAnalyzer().polarity_scores(x)['compound'] if isinstance(x, str) else np.nan)
    elif algo == 'textblob':
        df_jobs_to_be_processed['sentiment'] = df_jobs_to_be_processed[text_col].progress_apply(lambda x: TextBlob(x).sentiment.polarity)
    ## rescaled
    if sentiment_range != (-1,1):
        df_jobs_to_be_processed['sentiment'] = preprocessing.MinMaxScaler(feature_range=sentiment_range).fit_transform(df_jobs_to_be_processed[['sentiment']])
    # print(df_jobs_to_be_processed[['sentiment']].describe().T)

    return df_jobs_to_be_processed


In [112]:
# Sentiment
from nltk.sentiment.vader import SentimentIntensityAnalyzer

vader = SentimentIntensityAnalyzer()

for sent in spacy_sentencized:
    print(vader.polarity_scores(sent)['compound'])


0.0
0.2023
0.0
0.0
0.4215
0.0
0.0
0.0
0.0
0.0
0.8442
0.3818
0.2023
0.4588
0.0
0.4019
0.0
0.0
0.5994
-0.2263
0.8126
0.0
0.5719
0.4588
0.34
0.3612
0.0
0.0
0.0
0.0
-0.128
-0.128
0.0
0.8402
0.4404
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.1779
0.4215
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.296
0.25
0.0
0.4404
0.3182
0.0
0.4404
0.0
0.743
0.0
0.0
0.4215
0.0
0.0
0.0
0.4019
0.0
0.0
0.0
0.4215
0.0
0.0
0.0
0.0
0.0
0.6369
0.0
0.0
0.5719
0.0
0.4019
0.0
0.0
0.0
0.5994
-0.2263
0.8807
0.0
0.5719
0.4588
0.34
0.3818
0.0
0.0
0.8225
0.6369
0.3818
0.0
0.4404
0.0
0.0
0.3818
0.4215
0.0
0.0
0.0
0.0
0.0
0.4215
0.0
0.0
0.0
0.0
0.0
0.296
0.0
0.0
0.4927
0.3182
0.3182
0.5859
0.0
0.4404
0.0
0.0
0.0
0.743
0.3182
0.0
0.0


In [110]:
# # Sentiment
# from textblob import TextBlob, Word

# for sent in spacy_sentencized:
#     print(TextBlob(sent).sentiment.polarity)


0.0
0.025
0.0
0.0
-0.10000000000000002
0.0
0.0
0.0
-0.08333333333333333
0.0
0.6666666666666666
0.4
-0.125
0.0
0.0
0.0
0.0
0.0
0.5
0.5
0.3833333333333333
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.45
0.0
0.0
0.0
0.0
-0.4
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.03333333333333333
0.1
0.0
0.0
0.0
-0.05555555555555555
0.0
0.0
0.2333333333333333
0.0
0.0
0.7
0.0
0.4333333333333333
0.03125
0.5
0.9
0.0
0.0
0.0
0.35714285714285715
0.175
0.0
0.0
-0.10000000000000002
0.0
0.0
0.35
0.0
0.0
0.10000000000000002
0.0
0.13636363636363635
0.375
0.0
0.0
0.0
0.0
0.0
0.5
0.5
0.2875
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.3333333333333333
0.5
0.06666666666666667
0.0
0.5
0.0
0.015625
0.015625
0.0
0.0
0.0
0.0
0.0
0.0
0.45
0.0
0.05
0.0
0.0
0.0
-0.05555555555555555
0.0
0.0
0.3033333333333333
0.0
-0.3
0.16
0.0
0.7
0.1
0.0
0.0
0.4333333333333333
0.0
0.0
0.0


In [ ]:
# Spacy sentencizer
# spacy_sentencized = []

if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    for job_description in job_descriptions:
        for token in nlp(job_description):
            print(token.text, token.tag_)
#             for sent in re.split(pattern, sentence.text):
#                 if len(sent) != 0:
#                     spacy_sentencized.append(sent)

In [ ]:
from spacy.matcher import Matcher
nlp = spacy.load('en_core_web_sm')

matcher = Matcher(nlp.vocab)

bigram_rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

trigram_patterns = [[{"POS": i} for i in j] for j in rules]


In [ ]:
trigram_patterns

In [ ]:
patterns = {
    'noun_verb': [{'POS': 'NOUN'}, {'POS': 'VERB'}],
    'verb_noun': [{'POS': 'VERB'}, {'POS': 'NOUN'}],
    'adj_noun': [{'POS': 'ADJ'}, {'POS': 'NOUN'}],
    'adj_propn': [{'POS': 'ADJ'}, {'POS': 'PROPN'}],
}

In [ ]:
for pattern_name, pattern in patterns.items():
    matcher.add(pattern_name, [pattern])


In [ ]:
for job_description in job_descriptions:
    doc = nlp(job_description)
    matches = matcher(doc)

In [ ]:
for match_id, start, end in matches:
    
    # Get string representation
    string_id = nlp.vocab.strings[match_id]

    # The matched span
    span = doc[start:end]
    
    print(repr(span.text))
    print(match_id, string_id, start, end)
    print()

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


In [ ]:
# for sentence in spacy_sentencized:
#     print(model.encode(sentence))



In [ ]:
from transformers.models.electra.modeling_tf_electra import TFElectraMainLayer

In [ ]:
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("pavanchhatpar/electra-base-sentence-splitter")

model = AutoModelForPreTraining.from_pretrained("pavanchhatpar/electra-base-sentence-splitter", from_tf=True)


In [ ]:
nlp = pipeline('sentence-splitter', model)